In [1]:
import sys
sys.path.append("..")

In [2]:
import bytelevel

In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from nltk import word_tokenize


In [5]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

Using TensorFlow backend.


In [20]:
import bowizer
import tfidf

In [7]:
with open('../data/norvig/big.txt') as f:
    data = f.read()

In [8]:
chunks = make_chunks(data)

In [9]:
r = np.random.RandomState(42)

In [10]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [11]:
def onehot(a, vocab_size=256):
    m, n = a.shape
    temp = np.zeros((m, n, vocab_size))
    temp[np.expand_dims(np.arange(m), axis=0).reshape(m, 1), np.repeat(np.array([np.arange(n)]), m, axis=0), a] = 1
    return temp



In [12]:
three_lines = """Midway upon the journey of our life
I found myself within a forest dark,
For the straightforward pathway had been lost."""
lines = three_lines.split('\n')


In [13]:
lines

['Midway upon the journey of our life',
 'I found myself within a forest dark,',
 'For the straightforward pathway had been lost.']

In [15]:
text = """Midway upon the journey of our life
I found myself within a forest dark,
For the straightforward pathway had been lost.

Ah me! how hard a thing it is to say
What was this forest savage, rough, and stern,
Which in the very thought renews the fear.

So bitter is it, death is little more;
But of the good to treat, which there I found,
Speak will I of the other things I saw there.

I cannot well repeat how there I entered,
So full was I of slumber at the moment
In which I had abandoned the true way.

But after I had reached a mountain's foot,
At that point where the valley terminated,
Which had with consternation pierced my heart,

Upward I looked, and I beheld its shoulders,
Vested already with that planet's rays
Which leadeth others right by every road.

Then was the fear a little quieted
That in my heart's lake had endured throughout
The night, which I had passed so piteously.

And even as he, who, with distressful breath,
Forth issued from the sea upon the shore,
Turns to the water perilous and gazes;

So did my soul, that still was fleeing onward,
Turn itself back to re-behold the pass
Which never yet a living person left.

After my weary body I had rested,
The way resumed I on the desert slope,
So that the firm foot ever was the lower."""

In [16]:
tokenized_text = bowizer.word_tokenize(text)
vocab_size = 5
extendVocabList, td, vocab = bowizer.get_vocab(tokenized_text, vocab_size)

In [49]:
class TokenMaker(object):
    def __init__(self, corpus, vocab_size, should_lower=True, unk='unk'):
        self.unk = unk
        self.should_lower = should_lower
        tokenized_text = bowizer.tokenize_docs(lines)
        tokenized_text = self.canonical(tokenized_text)
        self.vocab_size = vocab_size
        self.extendVocabList, self.td, self.vocab = bowizer.get_vocab(tokenized_text, vocab_size, unk=self.unk)
        
    def canonical(self, words):
        if self.should_lower:
            return [word.lower() for word in words]
        return words
    
    def vector(self, text):
        words = bowizer.word_tokenize(text)
        words = [tfidf.toUnk(word, self.vocab, self.unk) for word in words]
        return [self.td[word] for word in words]
    
    def x(self, lines, maxlen=None):
        vectors = [self.vector(line) for line in lines]
        return pad_sequences(vectors, maxlen=maxlen)
    
    def y(self, x):
        return onehot(x, self.vocab_size + 1)

In [51]:
tm = TokenMaker([text], 20)

In [52]:
tm.vector(three_lines)

[0, 3, 1, 4, 5, 6, 7, 0, 9, 10, 11, 12, 13, 14, 15, 0, 1, 17, 18, 19, 20, 0, 0]

In [53]:
x = tm.x(lines, maxlen=30)

In [54]:
tm.y(x)

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]])

In [62]:
class SlicedWordData(object):
    def __init__(self, lines, max_len, tokenmaker):
        self.line = lines
        self.max_len = max_len
        self.tokenmaker = tokenmaker
        self.x = self.tokenmaker.x(lines)
        self.y = self.tokenmaker.y(self.x)
        
    @staticmethod
    def Random(train_text, test_text, max_len, n, r):
        def random_slice(data):
            i = r.randint(len(data) - max_len)
            return data[i : i + max_len]
        train_slices = [random_slice(train_text) for _ in range(n)]
        test_slices = [random_slice(test_text) for _ in range(int(0.1 * n))]

        return SlicedData(train_slices, max_len), SlicedData(test_slices, max_len)



In [63]:
lines

['Midway upon the journey of our life',
 'I found myself within a forest dark,',
 'For the straightforward pathway had been lost.']

In [64]:
data = SlicedWordData(lines, 5, tm)

In [68]:
data.x.shape

(3, 8)

In [67]:
data.y.shape

(3, 8, 21)

In [11]:
foo = SlicedWordData(test_text, 32)

In [20]:
foo.x[1000:1002]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0, 104],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 101]], dtype=int32)

In [19]:
embed_h = EmbeddingHyper(256, 64)
conv_h = ConvHyper(128, 6, 4)
rnn_h = RnnHyper(256, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [20]:
dernn_h = RnnHyper(64, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(128, 6, 4)
decoder_h = td.Hyper(256, [dernn_h, dec_h])

In [37]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [38]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [39]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

In [40]:
models = {64: model64, 128: model128, 256: model256 }

In [41]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=100,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [42]:
history = None

In [43]:
for i in range(500):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 13s - loss: 3.4422 - categorical_accuracy: 0.1547 - val_loss: 3.1157 - val_categorical_accuracy: 0.1632
Epoch 2/5
10000/10000 [==============================] - 11s - loss: 3.1328 - categorical_accuracy: 0.1587 - val_loss: 3.0910 - val_categorical_accuracy: 0.1629
Epoch 3/5
10000/10000 [==============================] - 11s - loss: 3.1044 - categorical_accuracy: 0.1591 - val_loss: 3.0773 - val_categorical_accuracy: 0.1632
Epoch 4/5
10000/10000 [==============================] - 11s - loss: 3.0989 - categorical_accuracy: 0.1592 - val_loss: 3.0744 - val_categorical_accuracy: 0.1632
Epoch 5/5
10000/10000 [==============================] - 12s - loss: 3.0881 - categorical_accuracy: 0.1592 - val_loss: 3.0606 - val_categorical_accuracy: 0.1632
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 12s - loss: 3.0683 - categorical_ac

10000/10000 [==============================] - 11s - loss: 1.5900 - categorical_accuracy: 0.4998 - val_loss: 1.6219 - val_categorical_accuracy: 0.4907
19
Train on 10000 samples, validate on 1000 samples
Epoch 96/100
10000/10000 [==============================] - 11s - loss: 1.6058 - categorical_accuracy: 0.4952 - val_loss: 1.6031 - val_categorical_accuracy: 0.4977
Epoch 97/100
10000/10000 [==============================] - 11s - loss: 1.5771 - categorical_accuracy: 0.5037 - val_loss: 1.5882 - val_categorical_accuracy: 0.5012
Epoch 98/100
10000/10000 [==============================] - 11s - loss: 1.5586 - categorical_accuracy: 0.5088 - val_loss: 1.5757 - val_categorical_accuracy: 0.5054
Epoch 99/100
10000/10000 [==============================] - 11s - loss: 1.5478 - categorical_accuracy: 0.5121 - val_loss: 1.5657 - val_categorical_accuracy: 0.5064
Epoch 100/100
10000/10000 [==============================] - 11s - loss: 1.5308 - categorical_accuracy: 0.5167 - val_loss: 1.5647 - val_categ

10000/10000 [==============================] - 11s - loss: 0.9255 - categorical_accuracy: 0.7017 - val_loss: 0.9384 - val_categorical_accuracy: 0.6970
Epoch 188/190
10000/10000 [==============================] - 11s - loss: 0.9155 - categorical_accuracy: 0.7051 - val_loss: 0.9596 - val_categorical_accuracy: 0.6907
Epoch 189/190
10000/10000 [==============================] - 11s - loss: 0.9067 - categorical_accuracy: 0.7079 - val_loss: 0.9353 - val_categorical_accuracy: 0.6988
Epoch 190/190
10000/10000 [==============================] - 11s - loss: 0.9004 - categorical_accuracy: 0.7097 - val_loss: 0.9241 - val_categorical_accuracy: 0.6996
38
Train on 10000 samples, validate on 1000 samples
Epoch 191/195
10000/10000 [==============================] - 11s - loss: 0.9325 - categorical_accuracy: 0.6988 - val_loss: 0.9361 - val_categorical_accuracy: 0.6970
Epoch 192/195
10000/10000 [==============================] - 11s - loss: 0.9106 - categorical_accuracy: 0.7063 - val_loss: 0.9300 - val_c

10000/10000 [==============================] - 11s - loss: 0.6835 - categorical_accuracy: 0.7803 - val_loss: 0.7167 - val_categorical_accuracy: 0.7670
Epoch 280/280
10000/10000 [==============================] - 11s - loss: 0.6789 - categorical_accuracy: 0.7812 - val_loss: 0.7048 - val_categorical_accuracy: 0.7717
56
Train on 10000 samples, validate on 1000 samples
Epoch 281/285
10000/10000 [==============================] - 11s - loss: 0.7112 - categorical_accuracy: 0.7713 - val_loss: 0.7057 - val_categorical_accuracy: 0.7732
Epoch 282/285
10000/10000 [==============================] - 11s - loss: 0.6927 - categorical_accuracy: 0.7778 - val_loss: 0.7038 - val_categorical_accuracy: 0.7747
Epoch 283/285
10000/10000 [==============================] - 11s - loss: 0.6773 - categorical_accuracy: 0.7826 - val_loss: 0.7112 - val_categorical_accuracy: 0.7723
Epoch 284/285
10000/10000 [==============================] - 11s - loss: 0.6752 - categorical_accuracy: 0.7831 - val_loss: 0.7041 - val_c

10000/10000 [==============================] - 11s - loss: 0.5772 - categorical_accuracy: 0.8160 - val_loss: 0.5868 - val_categorical_accuracy: 0.8133
Epoch 372/375
10000/10000 [==============================] - 11s - loss: 0.5568 - categorical_accuracy: 0.8227 - val_loss: 0.5768 - val_categorical_accuracy: 0.8156
Epoch 373/375
10000/10000 [==============================] - 11s - loss: 0.5398 - categorical_accuracy: 0.8288 - val_loss: 0.5621 - val_categorical_accuracy: 0.8210
Epoch 374/375
10000/10000 [==============================] - 11s - loss: 0.5290 - categorical_accuracy: 0.8330 - val_loss: 0.5643 - val_categorical_accuracy: 0.8198
Epoch 375/375
10000/10000 [==============================] - 11s - loss: 0.5203 - categorical_accuracy: 0.8360 - val_loss: 0.5638 - val_categorical_accuracy: 0.8193
75
Train on 10000 samples, validate on 1000 samples
Epoch 376/380
10000/10000 [==============================] - 11s - loss: 0.5686 - categorical_accuracy: 0.8193 - val_loss: 0.5717 - val_c

...

10000/10000 [==============================] - 11s - loss: 0.1464 - categorical_accuracy: 0.9607 - val_loss: 0.1682 - val_categorical_accuracy: 0.9521
433
Train on 10000 samples, validate on 1000 samples
Epoch 2166/2170
10000/10000 [==============================] - 11s - loss: 0.1708 - categorical_accuracy: 0.9523 - val_loss: 0.1685 - val_categorical_accuracy: 0.9522
Epoch 2167/2170
10000/10000 [==============================] - 11s - loss: 0.1592 - categorical_accuracy: 0.9564 - val_loss: 0.1693 - val_categorical_accuracy: 0.9523
Epoch 2168/2170
10000/10000 [==============================] - 11s - loss: 0.1544 - categorical_accuracy: 0.9581 - val_loss: 0.1706 - val_categorical_accuracy: 0.9519
Epoch 2169/2170
10000/10000 [==============================] - 11s - loss: 0.1502 - categorical_accuracy: 0.9596 - val_loss: 0.1683 - val_categorical_accuracy: 0.9519
Epoch 2170/2170
10000/10000 [==============================] - 11s - loss: 0.1481 - categorical_accuracy: 0.9603 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1887 - categorical_accuracy: 0.9467 - val_loss: 0.2069 - val_categorical_accuracy: 0.9404
Epoch 2212/2215
10000/10000 [==============================] - 11s - loss: 0.3617 - categorical_accuracy: 0.8999 - val_loss: 0.2069 - val_categorical_accuracy: 0.9387
Epoch 2213/2215
10000/10000 [==============================] - 11s - loss: 0.1696 - categorical_accuracy: 0.9525 - val_loss: 0.1736 - val_categorical_accuracy: 0.9512
Epoch 2214/2215
10000/10000 [==============================] - 11s - loss: 0.1529 - categorical_accuracy: 0.9584 - val_loss: 0.1672 - val_categorical_accuracy: 0.9531
Epoch 2215/2215
10000/10000 [==============================] - 11s - loss: 0.1470 - categorical_accuracy: 0.9606 - val_loss: 0.1654 - val_categorical_accuracy: 0.9539
443
Train on 10000 samples, validate on 1000 samples
Epoch 2216/2220
10000/10000 [==============================] - 11s - loss: 0.1724 - categorical_accuracy: 0.9518 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1610 - categorical_accuracy: 0.9555 - val_loss: 0.1676 - val_categorical_accuracy: 0.9526
Epoch 2304/2305
10000/10000 [==============================] - 11s - loss: 0.1515 - categorical_accuracy: 0.9591 - val_loss: 0.1647 - val_categorical_accuracy: 0.9533
Epoch 2305/2305
10000/10000 [==============================] - 11s - loss: 0.1472 - categorical_accuracy: 0.9605 - val_loss: 0.1624 - val_categorical_accuracy: 0.9542
461
Train on 10000 samples, validate on 1000 samples
Epoch 2306/2310
10000/10000 [==============================] - 11s - loss: 0.1736 - categorical_accuracy: 0.9511 - val_loss: 0.1732 - val_categorical_accuracy: 0.9509
Epoch 2307/2310
10000/10000 [==============================] - 11s - loss: 0.1605 - categorical_accuracy: 0.9557 - val_loss: 0.1680 - val_categorical_accuracy: 0.9529
Epoch 2308/2310
10000/10000 [==============================] - 11s - loss: 0.1531 - categorical_accuracy: 0.9582 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1500 - categorical_accuracy: 0.9593 - val_loss: 0.1684 - val_categorical_accuracy: 0.9529
479
Train on 10000 samples, validate on 1000 samples
Epoch 2396/2400
10000/10000 [==============================] - 11s - loss: 0.1713 - categorical_accuracy: 0.9518 - val_loss: 0.1719 - val_categorical_accuracy: 0.9519
Epoch 2397/2400
10000/10000 [==============================] - 11s - loss: 0.1574 - categorical_accuracy: 0.9566 - val_loss: 0.1691 - val_categorical_accuracy: 0.9528
Epoch 2398/2400
10000/10000 [==============================] - 11s - loss: 0.1508 - categorical_accuracy: 0.9592 - val_loss: 0.1683 - val_categorical_accuracy: 0.9530
Epoch 2399/2400
10000/10000 [==============================] - 11s - loss: 0.1467 - categorical_accuracy: 0.9606 - val_loss: 0.1673 - val_categorical_accuracy: 0.9530
Epoch 2400/2400
10000/10000 [==============================] - 12s - loss: 0.1442 - categorical_accuracy: 0.9613 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1719 - categorical_accuracy: 0.9511 - val_loss: 0.1720 - val_categorical_accuracy: 0.9516
Epoch 2442/2445
10000/10000 [==============================] - 11s - loss: 0.1587 - categorical_accuracy: 0.9558 - val_loss: 0.1693 - val_categorical_accuracy: 0.9521
Epoch 2443/2445
10000/10000 [==============================] - 11s - loss: 0.1534 - categorical_accuracy: 0.9576 - val_loss: 0.1676 - val_categorical_accuracy: 0.9526
Epoch 2444/2445
10000/10000 [==============================] - 11s - loss: 0.1491 - categorical_accuracy: 0.9591 - val_loss: 0.1680 - val_categorical_accuracy: 0.9531
Epoch 2445/2445
10000/10000 [==============================] - 11s - loss: 0.1478 - categorical_accuracy: 0.9594 - val_loss: 0.1728 - val_categorical_accuracy: 0.9506
489
Train on 10000 samples, validate on 1000 samples
Epoch 2446/2450
10000/10000 [==============================] - 11s - loss: 0.2964 - categorical_accuracy: 0.9210 - val_loss: 0.4

In [45]:
train, test = SlicedData.Random(train_text, test_text, 64, 10000, r)
test.text[:4]

['I do not yet know. There will be time enough to\nthink about love',
 'ed in America and propagated in France with so\nmuch enthusiasm a',
 'or two persons without indicating which office each was to fill,',
 ' has happened or to hide it\nfrom him are both equally impossible']

In [47]:
bytelevel.prediction2str(model64.predict(test.x[:4]))

['I do not yet know. There will be time enough to\nthinksabout ron ',
 'ed in America and propagated in France with so\nmuch enthusias  o',
 'or two persons without indicating which office each was to fine ',
 ' has happened or to hide it\nfrom him are bothsequally impossine ']

In [106]:
model64.save('../models/model64.h5')
with open('../models/model64.pkl', 'wb') as f:
    pickle.dump([])